In [5]:
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import requests
import time
import pickle
from config import *
from prettytable import PrettyTable

# Set up Spotify API client

auth_manager = SpotifyClientCredentials(client_id=client_id,
                                         client_secret=client_secret_id)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [6]:
songs_df = pd.read_csv("songs1.csv")

In [7]:
songs_df

,title,artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,...,id,uri,track_href,analysis_url,duration_ms,time_signature,open_url,dataset,cluster,cluster2
0,Kill Bill,SZA,0.644,0.728,8,-5.750,1,0.0351,0.054300,0.169000,...,3OHfY25tqY28d16oZczHc8,spotify:track:3OHfY25tqY28d16oZczHc8,https://api.spotify.com/v1/tracks/3OHfY25tqY28...,https://api.spotify.com/v1/audio-analysis/3OHf...,153947,4,https://open.spotify.com/track/3OHfY25tqY28d16...,hot_songs,1,5
1,Last Night,Morgan Wallen,0.502,0.667,6,-5.486,1,0.0342,0.445000,0.000000,...,6Hiz5MDYoGdLBBQ3gGzRam,spotify:track:6Hiz5MDYoGdLBBQ3gGzRam,https://api.spotify.com/v1/tracks/6Hiz5MDYoGdL...,https://api.spotify.com/v1/audio-analysis/6Hiz...,163855,4,https://open.spotify.com/track/6Hiz5MDYoGdLBBQ...,hot_songs,4,4
2,Flowers,Miley Cyrus,0.707,0.681,0,-4.325,1,0.0668,0.063200,0.000005,...,4DHcnVTT87F0zZhRPYmZ3B,spotify:track:4DHcnVTT87F0zZhRPYmZ3B,https://api.spotify.com/v1/tracks/4DHcnVTT87F0...,https://api.spotify.com/v1/audio-analysis/4DHc...,200455,4,https://open.spotify.com/track/4DHcnVTT87F0zZh...,hot_songs,1,2
3,Princess Diana,Ice Spice & Nicki Minaj,0.898,0.676,9,-5.196,1,0.1870,0.140000,0.000000,...,0ZxhtATQ0a6EvQNZ0z6NKV,spotify:track:0ZxhtATQ0a6EvQNZ0z6NKV,https://api.spotify.com/v1/tracks/0ZxhtATQ0a6E...,https://api.spotify.com/v1/audio-analysis/0Zxh...,172125,4,https://open.spotify.com/track/0ZxhtATQ0a6EvQN...,hot_songs,1,5
4,Ella Baila Sola,Eslabon Armado X Peso Pluma,0.668,0.758,5,-5.176,0,0.0332,0.483000,0.000019,...,3dnP0JxCgygwQH9Gm7q7nb,spotify:track:3dnP0JxCgygwQH9Gm7q7nb,https://api.spotify.com/v1/tracks/3dnP0JxCgygw...,https://api.spotify.com/v1/audio-analysis/3dnP...,165671,3,https://open.spotify.com/track/3dnP0JxCgygwQH9...,hot_songs,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3081,Skank and Dub,Junior Reid,0.969,0.870,9,-4.805,1,0.1230,0.004160,0.005930,...,1Ks2UH3WvpohH9q8b03NM7,spotify:track:1Ks2UH3WvpohH9q8b03NM7,https://api.spotify.com/v1/tracks/1Ks2UH3Wvpoh...,https://api.spotify.com/v1/audio-analysis/1Ks2...,200742,4,https://open.spotify.com/track/1Ks2UH3WvpohH9q...,not_hot_songs,1,5
3082,I Guess I'm Crazy,Jim Reeves,0.497,0.261,4,-13.869,1,0.0273,0.761000,0.000291,...,4GPbOUUbitW3e6aEVccrhf,spotify:track:4GPbOUUbitW3e6aEVccrhf,https://api.spotify.com/v1/tracks/4GPbOUUbitW3...,https://api.spotify.com/v1/audio-analysis/4GPb...,142153,3,https://open.spotify.com/track/4GPbOUUbitW3e6a...,not_hot_songs,2,6
3083,Stop Breaking Down (1994 Digital Remaster),The Rolling Stones,0.522,0.891,9,-4.085,1,0.0322,0.509000,0.908000,...,09eg3FGzsZQbZk8qEXhJIA,spotify:track:09eg3FGzsZQbZk8qEXhJIA,https://api.spotify.com/v1/tracks/09eg3FGzsZQb...,https://api.spotify.com/v1/audio-analysis/09eg...,274773,4,https://open.spotify.com/track/09eg3FGzsZQbZk8...,not_hot_songs,1,5
3084,Albo,Hevia,0.594,0.284,0,-16.440,0,0.0309,0.438000,0.876000,...,2lAzmBq6991fuq3DbcL2hN,spotify:track:2lAzmBq6991fuq3DbcL2hN,https://api.spotify.com/v1/tracks/2lAzmBq6991f...,https://api.spotify.com/v1/audio-analysis/2lAz...,140671,4,https://open.spotify.com/track/2lAzmBq6991fuq3...,not_hot_songs,0,0


In [ ]:
# Function to get song features from Spotify API
def get_song_features(title, artist):
    """
    Gets the audio features of a song from the Spotify API
    given its title and artist name.
    """
    results = sp.search(q=f"{title} artist:{artist}", type="track", limit=1)
    if results["tracks"]["total"] > 0:
        track_id = results["tracks"]["items"][0]["id"]
        track_features = sp.audio_features([track_id])[0]
        return track_features
    else:
        return None



In [2]:
def run_recommender():
    title = input ("Enter song: ")
    artist = input("Enter artist: ")
 
    audio_feat = get_song_features(title, artist)
        
    d = {
        "danceability" : [audio_feat["danceability"]],
        "energy" : [audio_feat["energy"]],
        "loudness": [audio_feat["loudness"]],
        "speechiness" : [audio_feat["speechiness"]],
        "acousticness" : [audio_feat["acousticness"]],
        "instrumentalness":  [audio_feat["instrumentalness"]],
        "liveness" : [audio_feat["liveness"]],
        "valence"  : [audio_feat["valence"]],
        "tempo" : [audio_feat["tempo"]],
        "key" : [audio_feat["key"]]

    }
    
    with open("scaler.pickle", "rb") as f:
        stdScaler = pickle.load(f)
        
    feat_scaled = stdScaler.transform(pd.DataFrame(d))
    feat_scaled = pd.DataFrame(feat_scaled, columns = d.keys())

    with open("kmeans_7.pickle", "rb") as f:
        kmeans = pickle.load(f)
        
    cluster = kmeans.predict(feat_scaled)[0]
    print(f"Cluster {cluster}")
    
    res = songs_df[(songs_df["artists"] == artist) &\
                   (songs_df["title"] == title)]
    if not res.empty:
        # Title/Artist already exists in our songDB
        hot_or_not = res["dataset"]
        c = res["cluster2"]
        
        rec_song = songs_df[(songs_df["dataset"] == hot_or_not) & \
                 (songs_df["cluster2"] == c)].sample()[["title", "artists", "open_url"]]
    else:
        rec_song = songs_df[(songs_df["cluster2"] == cluster) & \
                 (songs_df["dataset"] == "not_hot_songs")].sample()[["title", "artists", "open_url"]]
        
    print("Title: {}\nArtist: {}\nLink: {}".format(rec_song['title'].values[0], rec_song['artists'].values[0], rec_song['open_url'].values[0]))


In [22]:
def more_recommendations():
    table = PrettyTable()
    while True:
        run_recommender()
    
        quit = input("Wanne quit? (y/n) ")
        if quit == "y":
            break

    print("Good bye!")

In [23]:
more_recommendations()

Enter song: kill bill
Enter artist: 


C:\Users\celia\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but KMeans was fitted without feature names
  warnings.warn(


Cluster 3
Title: informa
Artist: Elephant Man
Link: https://open.spotify.com/track/46l7fOJggsNqrNPJeFk0og
Wanne quit? (y/n) y
Good bye!
